In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import numpy as np
import cv2
import os
import random
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import tensorflow
from tensorflow import keras
import glob
import shutil
from PIL import Image
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn import preprocessing
from keras.applications.vgg19 import VGG19, preprocess_input
from keras.applications import ResNet50
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.layers import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import mean_absolute_error
from sklearn.utils.class_weight import compute_class_weight
from keras.applications.inception_v3 import decode_predictions
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import AUC

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
dataset_path = '/kaggle/input/covid19-radiography-dataset/COVID-19_Radiography_Dataset/COVID-19_Radiography_Dataset/COVID'
working_path = '/kaggle/working/Covid'

# Ensure the destination directory exists
os.makedirs(working_path, exist_ok=True)

# List all files in the dataset folder
all_images = os.listdir(dataset_path)

# Shuffle the list of images to select a random subset
random.shuffle(all_images)

# Select the first 500 images
selected_images = all_images[:1000]

# Copy the selected images to the working directory
for image in selected_images:
    source_path = os.path.join(dataset_path, image)
    img = cv2.imread(source_path)
    
    # Resize the image to 224x224
    img = cv2.resize(img, (224, 224))
    
    # Save the resized image to the working directory
    destination_path = os.path.join(working_path, image)
    cv2.imwrite(destination_path, img)

In [5]:
csv_file_path = '/kaggle/input/nih-chest-x-ray-14-224x224-resized/Data_Entry_2017.csv'
source_images_path = '/kaggle/input/nih-chest-x-ray-14-224x224-resized/images-224/images-224'
destination_path = '/kaggle/working/'

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file_path)

# Define the classes you want to extract and the number of samples per class
classes_to_extract = ['Atelectasis', 'Consolidation', 'Infiltration', 'Effusion', 'Pneumothorax']
samples_per_class = 1000

# Create the destination folders for each class
for class_name in classes_to_extract:
    os.makedirs(os.path.join(destination_path, class_name), exist_ok=True)

# Initialize a counter for each class
class_counters = {class_name: 0 for class_name in classes_to_extract}

# Loop through the rows of the DataFrame and copy the images to the destination folders
for index, row in df.iterrows():
    image_label = row['Finding Labels']  # Adjust 'label' based on your CSV column name

    # Check if the image belongs to one of the classes you want to extract
    if image_label in classes_to_extract and class_counters[image_label] < samples_per_class:
        image_filename = row['Image Index']  # Adjust 'filename' based on your CSV column name
        source_image_path = os.path.join(source_images_path, image_filename)
        destination_image_path = os.path.join(destination_path, image_label, image_filename)

        # Copy the image to the destination folder
        shutil.copyfile(source_image_path, destination_image_path)

        # Update the counter for the class
        class_counters[image_label] += 1

print("Extraction completed.")

Extraction completed.


In [2]:
Directory = r"/kaggle/working/"
#arranging the list based on the folders saved in Directory
Categories = ["Normal","Covid","Pneumonia","Atelectasis","Effusion","Pneumothorax","Consolidation","Infiltration"]

In [3]:
data = []

for category in Categories:
    path = os.path.join(Directory, category)
    label = Categories.index(category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path, img))
        img_array = cv2.resize(img_array, (224, 224))  # Use the original size (224, 224)
        data.append([img_array, label])

In [4]:
random.shuffle(data)

In [5]:
X = []
y = []

for features, labels in data:
    X.append(features)
    y.append(labels)

In [6]:
X = np.array(X)
y = np.array(y)

In [7]:
X.shape

(8000, 224, 224, 3)

In [7]:
X = X/255

In [11]:
X[0]

array([[[0.16078431, 0.16078431, 0.16078431],
        [0.10588235, 0.10588235, 0.10588235],
        [0.05882353, 0.05882353, 0.05882353],
        ...,
        [0.43921569, 0.43921569, 0.43921569],
        [0.43529412, 0.43529412, 0.43529412],
        [0.42745098, 0.42745098, 0.42745098]],

       [[0.09411765, 0.09411765, 0.09411765],
        [0.03921569, 0.03921569, 0.03921569],
        [0.00784314, 0.00784314, 0.00784314],
        ...,
        [0.36862745, 0.36862745, 0.36862745],
        [0.40784314, 0.40784314, 0.40784314],
        [0.43921569, 0.43921569, 0.43921569]],

       [[0.07843137, 0.07843137, 0.07843137],
        [0.02352941, 0.02352941, 0.02352941],
        [0.        , 0.        , 0.        ],
        ...,
        [0.27058824, 0.27058824, 0.27058824],
        [0.34509804, 0.34509804, 0.34509804],
        [0.43921569, 0.43921569, 0.43921569]],

       ...,

       [[0.55686275, 0.55686275, 0.55686275],
        [0.56078431, 0.56078431, 0.56078431],
        [0.56470588, 0

In [8]:
#Using Pretrained Model
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224,224,3))

In [9]:
len(base_model.layers)

22

In [10]:
#Fine Tuning
#1) Freezing the model layers
for layer in base_model.layers[:23]:
    layer.trainable = False

In [11]:
#2) adding custom layers on top
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)         #increase the neurons (from 60 it went 62.)
#x = Dropout(0.5)(x)
#x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
output_layer = Dense(len(Categories), activation='softmax')(x)

In [12]:
#3) creating the model
from tensorflow.keras.models import Model
model = Model(inputs=base_model.input, outputs=output_layer)

In [13]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [14]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [15]:
model.fit(X, y, batch_size=32,
          epochs=5, validation_split=0.1)

Epoch 1/5
225/225 [==============================] - 49s 181ms/step - loss: 1.5475 - accuracy: 0.4621 - val_loss: 1.1470 - val_accuracy: 0.5425
Epoch 2/5
225/225 [==============================] - 41s 184ms/step - loss: 1.0655 - accuracy: 0.5807 - val_loss: 1.0529 - val_accuracy: 0.5788
Epoch 3/5
225/225 [==============================] - 41s 181ms/step - loss: 1.0036 - accuracy: 0.6017 - val_loss: 0.9890 - val_accuracy: 0.6025
Epoch 4/5
225/225 [==============================] - 41s 181ms/step - loss: 0.9196 - accuracy: 0.6376 - val_loss: 0.9617 - val_accuracy: 0.6225
Epoch 5/5
225/225 [==============================] - 41s 181ms/step - loss: 0.8606 - accuracy: 0.6621 - val_loss: 0.9361 - val_accuracy: 0.6363


In [ ]:
pickle.dump(model,open("VGG19Model.h5","wb"))

In [ ]:
model = pickle.load(open("","rb"))

In [ ]:
image = cv2.imread("/kaggle/input/covid19-radiography-dataset/COVID-19_Radiography_Dataset/COVID-19_Radiography_Dataset/Lung_Opacity/Lung_Opacity-1011.png")

# Resize the image
image = cv2.resize(image, (224, 224))

# Convert the image to grayscale

# Normalize pixel values to the range [0, 1]
image = image / 255.0

# Expand dimensions to match the input shape expected by the model
image = np.expand_dims(image, axis=0)
# Convert to numpy array (optional, as the previous line already returns a NumPy array)
image = np.array(image)


In [ ]:
diseases = ["Pneumothorax","Normal","Infiltration","Effusion","Atelectasis","Covid","Pneumonia","Consolidation"]
predictions = model.predict(image)

# Print the top 5 predictions for each sample in the test data
top3_indices = np.argsort(predictions[0])[::-1][:3]  # Get indices of top 5 predictions
top3_classes = [f"Class {index + 1}" for index in top3_indices]
top3_probs = predictions[0][top3_indices]

highest_prob_index = top3_indices[0]
highest_prob_disease = diseases[highest_prob_index]

print(f"Given Image - Top 3 Predictions:")
for i in range(3):
    print(f"{top3_classes[i]}: {top3_probs[i]:.4f}")
print("\n")
print(f"Disease Predicted: {highest_prob_disease}")